**1. Importing required Library**

In [1]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator, image
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt


Using TensorFlow backend.


**2. Define some constant needed throughout the script**

In [2]:
N_CLASSES = 4
EPOCHS = 1000
PATIENCE = 5
TRAIN_PATH= '/floyd/input/tomato_disease_v3_mix/Train'
VALID_PATH = '/floyd/input/tomato_disease_v3_mix/Validation'
TEST_PATH = '/floyd/input/tomato_disease_v3_mix/test filtered'
MODEL_CHECK_WEIGHT_NAME = 'resnet_tomato_v3_chk.h5'

**3. Define model to be used**
we freeze the pre trained resnet model weight, and add few layer on top of it to utilize our custom dataset


In [3]:
K.set_learning_phase(0)
model = ResNet50(input_shape=(224,224,3),include_top=False, weights=None, pooling='avg')
K.set_learning_phase(1)
x = model.output
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASSES, activation='softmax', name='custom_output')(x)
custom_resnet = Model(inputs=model.input, outputs = output)

#for layer in model.layers:
#    layer.trainable = False

custom_resnet.compile(Adam(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
#custom_resnet.summary()

**4. Load dataset to be used**

In [4]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
traingen = datagen.flow_from_directory(TRAIN_PATH, target_size=(224,224), batch_size=32, class_mode='categorical')
validgen = datagen.flow_from_directory(VALID_PATH, target_size=(224,224), batch_size=32, class_mode='categorical')
testgen = datagen.flow_from_directory(TEST_PATH,target_size=(224,224), batch_size=32, class_mode='categorical', shuffle=False )

Found 6750 images belonging to 4 classes.
Found 1198 images belonging to 4 classes.
Found 114 images belonging to 4 classes.


**5. Train Model**
we use ModelCheckpoint to save the best model based on validation accuracy


In [ ]:
es_callback = EarlyStopping(monitor='val_acc', patience=PATIENCE, mode='max')
mc_callback = ModelCheckpoint(filepath=MODEL_CHECK_WEIGHT_NAME, monitor='val_acc', save_best_only=True, mode='max')

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, mode='min', verbose = 1)
train_history = custom_resnet.fit_generator(traingen, steps_per_epoch=len(traingen), epochs= EPOCHS, validation_data=validgen, validation_steps=len(validgen), verbose=2, callbacks=[reduce])


Epoch 1/1000
 - 213s - loss: 11.1231 - acc: 0.3074 - val_loss: 10.8844 - val_acc: 0.3247
Epoch 2/1000
 - 193s - loss: 11.0411 - acc: 0.3150 - val_loss: 10.9382 - val_acc: 0.3214
Epoch 3/1000
 - 193s - loss: 11.0395 - acc: 0.3151 - val_loss: 10.6557 - val_acc: 0.3389
Epoch 4/1000
 - 192s - loss: 1.6220 - acc: 0.3891 - val_loss: 1.3214 - val_acc: 0.3548
Epoch 5/1000
 - 192s - loss: 1.2930 - acc: 0.3877 - val_loss: 1.3256 - val_acc: 0.3548
Epoch 6/1000
 - 192s - loss: 1.2892 - acc: 0.3903 - val_loss: 1.2930 - val_acc: 0.3548
Epoch 7/1000
 - 192s - loss: 1.2883 - acc: 0.3978 - val_loss: 1.2898 - val_acc: 0.3548
Epoch 8/1000
 - 192s - loss: 1.2883 - acc: 0.3997 - val_loss: 1.2832 - val_acc: 0.3548
Epoch 9/1000
 - 192s - loss: 1.2865 - acc: 0.3982 - val_loss: 1.2847 - val_acc: 0.3548
Epoch 10/1000
 - 192s - loss: 1.2894 - acc: 0.3973 - val_loss: 1.2943 - val_acc: 0.3548
Epoch 11/1000
 - 192s - loss: 1.2870 - acc: 0.3973 - val_loss: 1.2981 - val_acc: 0.3548
Epoch 12/1000
 - 192s - loss: 1.286

In [ ]:
plt.figure(1)
plt.subplot(221)
plt.plot(train_history.history['acc'])
plt.plot(train_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train accuracy','validation accuracy'])

plt.subplot(222)
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train loss', 'validation loss'])

plt.show()

**7. Load Last Checkpoint Weight**

In [9]:
#custom_resnet.load_weights(MODEL_CHECK_WEIGHT_NAME)

**7. Print validation confusion matrix, classification report, and accuracy**

In [10]:
predict = custom_resnet.predict_generator(testgen, steps=len(testgen), verbose=1)
test_labels = testgen.classes
confusion_matrix(test_labels, predict.argmax(axis=1))


5/5 [==============================] - 5s 920ms/step


array([[ 0,  0,  0,  0,  4,  9,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  5,  9,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  6,  8,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  4, 10,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  3, 10],
       [ 9,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 4, 11,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  4,  9,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  3,  9,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  6,  8,  0,  0,  0,  0,  0]], dtype=int64)

In [11]:
cr_labels = list(testgen.class_indices.keys())
classification_report(test_labels, predict.argmax(axis=1), target_names=cr_labels)

'                           precision    recall  f1-score   support\n\n              bald_uakari       0.00      0.00      0.00        13\nblack_headed_night_monkey       0.00      0.00      0.00        14\n   common_squirrel_monkey       0.00      0.00      0.00        14\n         japanese_macaque       0.00      0.00      0.00        15\n           mantled_howler       0.00      0.00      0.00        13\n           nilgiri_langur       0.00      0.00      0.00        13\n             patas_monkey       0.00      0.00      0.00        15\n           pygmy_marmoset       0.00      0.00      0.00        13\n         silvery_marmoset       0.00      0.00      0.00        13\n    white_headed_capuchin       0.00      0.00      0.00        14\n\n                micro avg       0.00      0.00      0.00       137\n                macro avg       0.00      0.00      0.00       137\n             weighted avg       0.00      0.00      0.00       137\n'

In [12]:
accuracy_score(test_labels,predict.argmax(axis=1))

0.0